In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
#export 
from exp.nb_02 import *
import torch.nn.functional as F

### Initial Setup

#### Data

In [3]:
mpl.rcParams['image.cmap'] = 'gray'

In [4]:
x_train, y_train, x_valid, y_valid = get_data()

In [5]:
n, m = x_train.shape
c = y_train.max()+1
nh = 50

In [6]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in,nh), nn.ReLU(), nn.Linear(nh,n_out)]
        
    def __call__(self, x):
        for l in self.layers: x = l(x)
        return x

In [7]:
model = Model(m, nh, 10)

In [8]:
pred = model(x_train)

#### Loss function: Cross entropy loss

Because multi-class label problem 

Output = preds

1. We take e^output for every class or label
2. We take the sum(e^outputs)
3. Prediction will be e^output/sum(e^outputs) - which adds up to 1

In [9]:
# Pytorch negative log-liklihood requires log softmax - hence the log
def log_softmax(x): return (x.exp()/(x.exp().sum(-1,keepdim=True))).log()

In [10]:
sm_pred = log_softmax(pred)

Essentially calculating negative log likelihood only requires the value of the target because the other part of the formula is just a multiplication with 0 - i.e. waste

The method below allows us to write negative log likelihood in a very concise manner

In [11]:
# explanation at Lesson 9 32 mins 
y_train[:3]

tensor([5, 0, 4])

In [12]:
# we actually only want the value in the 5th position because this is the target - see y_train above
sm_pred[0]

tensor([-2.2628, -2.2993, -2.4600, -2.5769, -2.2144, -2.2638, -2.3418, -2.1843,
        -2.1761, -2.3145], grad_fn=<SelectBackward>)

In [13]:
sm_pred[0][5]

tensor(-2.2638, grad_fn=<SelectBackward>)

In [14]:
# so to calculate the loss of the first 3 values we want the following indices
sm_pred[[0,1,2],[5,0,4]]

tensor([-2.2638, -2.3113, -2.2257], grad_fn=<IndexBackward>)

In [15]:
y_train.shape[0]

50000

In [16]:
def nll(input, target): return -input[range(target.shape[0]), target].mean() # range(target.shape[0]) selects all the rows

In [17]:
loss = nll(sm_pred, y_train)

In [18]:
loss

tensor(2.3165, grad_fn=<NegBackward>)

In [19]:
def log_softmax(x): return x - x.exp().sum(-1,keepdim=True).log()

In [20]:
test_near(nll(log_softmax(pred), y_train), loss)

We're using LogSumExp - when using e^something numbers can get very large - when using floating point numbers they tend to get less accurate as they get very large - two numbers 1000 appart will be treated similarly by computers - which is bad when doing gradient calculations

The trick below allows us to do the same calculation without the numerical problems of floating point calculations

In [21]:
# numerical stability trick
def logsumexp(x):
    m = x.max(-1)[0]
    return m + (x-m[:,None]).exp().sum(-1).log()    

In [22]:
test_near(logsumexp(pred), pred.logsumexp(-1))

In [23]:
def log_softmax(x): return x - x.logsumexp(-1, keepdim=True)

In [24]:
test_near(nll(log_softmax(pred), y_train), loss)

In [25]:
# Pytorch implementaion
test_near(F.nll_loss(F.log_softmax(pred, -1), y_train), loss)

#### Which is the same as cross_entropy loss

In [26]:
test_near(F.cross_entropy(pred, y_train), loss)

#### Basic training loop

The training loop will repeat over the following steps

- get the output of the model on a batch of inputs
- compare the output to the labels we have and compute a loss
- calculate the gradients of the loss with respect to every parameter of the model
- update the parameters with those gradients to make them a little bit better

In [27]:
loss_func = F.cross_entropy

We are grabbing the highest number from the output, with the index being the prediction and check if it's equal to the actual label

In [28]:
#export 
def accuracy(out, yb): return (torch.argmax(out,dim=1)==yb).float().mean()

In [29]:
bs=64 #batch size

xb = x_train[0:bs] # gets a mini-batch from x
preds = model(xb) # preds
preds[0], preds.shape

(tensor([ 0.0207, -0.0158, -0.1765, -0.2934,  0.0691,  0.0197, -0.0583,  0.0992,
          0.1074, -0.0310], grad_fn=<SelectBackward>), torch.Size([64, 10]))

In [30]:
yb = y_train[0:bs]
loss_func(preds,yb)

tensor(2.3217, grad_fn=<NllLossBackward>)

In [31]:
accuracy(preds,yb)

tensor(0.0938)

Accuracy is giving random answers because we have yet to train the model

In [32]:
def update():
    # calculate y_hat for linear model
    y_hat = x@a
    # calculate loss - compare actual with preds
    loss = mse(y, y_hat)
    # print loss every 10 steps
    if t % 10 == 0: print(loss)
    # calculate the gradients
    loss.backward()
    with torch.no_grad():
        # subtract learning rate * gradient from parameters
        a.sub_(lr* a.grad)
        # zero gradients
        a.grad.zero_()

In [33]:
lr = 0.5 # learning rate
epochs = 1 

In [34]:
model.layers

[Linear(in_features=784, out_features=50, bias=True),
 ReLU(),
 Linear(in_features=50, out_features=10, bias=True)]

In [35]:
model.layers[0]

Linear(in_features=784, out_features=50, bias=True)

In [36]:
model.layers[0].weight

Parameter containing:
tensor([[-0.0252,  0.0172, -0.0078,  ...,  0.0220, -0.0137,  0.0261],
        [-0.0084, -0.0175, -0.0071,  ...,  0.0012,  0.0263, -0.0118],
        [ 0.0219, -0.0201, -0.0089,  ..., -0.0081, -0.0106, -0.0074],
        ...,
        [ 0.0037, -0.0296, -0.0204,  ..., -0.0264, -0.0237,  0.0078],
        [-0.0038,  0.0181,  0.0270,  ..., -0.0292,  0.0166,  0.0039],
        [-0.0330, -0.0111,  0.0036,  ...,  0.0162,  0.0171, -0.0315]],
       requires_grad=True)

In [37]:
# gradient descent from scratch
for epoch in range(epochs):
    # calculates number of mini batches // double backslash returns floating point number from integer division 
    for i in range((n-1)//bs + 1): 
        start_i = i * bs # 0 64
        end_i = start_i + bs # 64 128
        loss = loss_func(model(xb), yb) # calculate loss on batch
        
        loss.backward() # calculate gradients
        with torch.no_grad(): # allows for in-place changing of parameters
            for l in model.layers:
                if hasattr(l, 'weight'):
                    l.weight -= l.weight.grad * lr
                    l.bias -= l.bias.grad * lr
                    l.weight.grad.zero_() # sets gradients to zero in place
                    l.bias.grad.zero_()                            
                    
                    # this is "slow" because we're going through each weight and bias individually

If we didn't zero the gradients after the update step, the algorithm would add the new gradients to the existing gradients.

Separating the update step from the zero-gradient step gives extra flexibility and provides an option to not zero the gradients for e.g. gradient accumulation - where you reduce the frequency of the optimizer step and zero.grad 

In [38]:
loss_func(model(xb), yb), accuracy(model(xb), yb)

(tensor(0.0006, grad_fn=<NllLossBackward>), tensor(1.))

Only ran for a mini-batch because <code>xb = xtrain[:bs] and bs=64</code>

### Parameters



In [39]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.l1 = nn.Linear(n_in, nh)
        self.l2 = nn.Linear(nh, n_out)
        
    def __call__(self, x): return self.l2(F.relu(self.l1(x)))        

In [40]:
x_train.shape

torch.Size([50000, 784])

In [41]:
m

784

In [42]:
model = Model(m, nh, 10) # 10 outputs

In [43]:
model.named_children().type

AttributeError: 'generator' object has no attribute 'type'

From Pytorch docs:

Returns an iterator over immediate children modules, yielding both
the name of the module as well as the module itself.

Yields:
(string, Module): Tuple containing a name and child module

In [ ]:
for name, l in model.named_children(): print(f'{name}: {l}')

In [ ]:
model

In [ ]:
model.l1

In [ ]:
def fit():
    for epoch in range(epochs): # loop through epochs
        for i in range((n-1)//bs + 1): 
            start_i = i * bs # 0 64
            end_i = start_i + bs # 64 128
            loss = loss_func(model(xb), yb) # calculate loss on batch
        
            loss.backward() # calculate gradients
            with torch.no_grad(): # allows for in-place changing of parameters
                for p in model.parameters(): p -= p.grad * lr  
                model.zero_grad()    

In [ ]:
fit()
loss_func(model(xb), yb), accuracy(model(xb), yb)

In [ ]:
accuracy(model(xb), yb)

super().\__init__ implements the functionality of the module below when we create a class (I think?)

on yield https://pythontips.com/2013/09/29/the-python-yield-keyword-explained/

In [ ]:
class DummyModule():
    def __init__(self, n_in, nh, n_out):
        self._modules = {}
        self.l1 = nn.Linear(n_in, nh)
        self.l2 = nn.Linear(nh, n_out)
        
    def __setattr__(self, k, v):
        if not k.startswith("_"): self._modules[k] = v # _modules gives it a name and turns it into a Pytorch nn.Module
        super().__setattr__(k,v) # make fn call itself on init
        
    def __repr__(self): return f'{self._modules}'
    
    def parameters(self):
        for l in self._modules.values():
            # yield speeds up computation because we dont want to save a copy to a list before it gets used 
            for p in l.paramters(): yield p      

In [ ]:
layers = [nn.Linear(m, nh), nn.ReLU(), nn.Linear(nh,10)]

In [ ]:
class Model(nn.Module):
    def __init__(self, layers):
        super().__init__()
        self.layers = layers
        for i,l in enumerate(self.layers): self.add_module(f'layer_{i}', l) # give it a name and add as Pythorch module
            
    def __call__(self, x):
        for l in self.layers: x = l(x)
        return x

In [ ]:
model = Model(layers)

In [ ]:
model

#### nn.ModuleList

In [ ]:
# Pytorch implementation
class SequentialModel(nn.Module):
    def __init__(self, layers):
        super().__init__()
        self.layers = nn.ModuleList(layers)
        
    def __call__(self, x):
        for l in self.layers: x = l(x)
        return x

In [ ]:
model = SequentialModel(layers)

In [ ]:
model

This is the equivalent to <code>nn.Sequential</code> from Pytorch

In [ ]:
class Optimizer():
    def __init__(self, params, lr=0.5): 
        self.params, self.lr=list(params), lr
    
    def step(self):
        with torch.no_grad():
            for p in self.params: p -= p.grad * lr
    
    # note how there is a data.zero vs. the model.zero_grad() in the fit fn above
    def zero_grad(self):
        for p in self.params: p.grad.data.zero_()

We're not using model.zero_grad() so that user can choose which gradients to optimize for e.g. gradual unfreezing or differential learning rates

In [ ]:
model = nn.Sequential(nn.Linear(m,nh), nn.ReLU(), nn.Linear(nh,10))

In [ ]:
opt = Optimizer(model.parameters())

In [ ]:
# now we can do:
for epoch in range(epochs): # loop through epochs
        for i in range((n-1)//bs + 1): 
            start_i = i * bs # 0 64
            end_i = start_i + bs # 64 128
            xb = x_train[start_i:end_i]
            yb = y_train[start_i:end_i]
            loss = loss_func(model(xb), yb) # calculate loss on batch
        
            loss.backward() # calculate gradients
            opt.step()
            opt.zero_grad()            

In [ ]:
loss, acc = loss_func(model(xb), yb), accuracy(model(xb), yb)
loss, acc

In [ ]:
#export 
from torch import optim

In [ ]:
optim.SGD.step?? # has some extra functionality like weight_decay, momentum, dampening? nesterov?

In [ ]:
# function to initialize model with SGD optimizer
def get_model():
    model = nn.Sequential(nn.Linear(m, nh), nn.ReLU(), nn.Linear(nh, 10))
    return model, optim.SGD(model.parameters(), lr=lr)

In [ ]:
model, opt = get_model() # init model
loss_func(model(xb), yb)

In [ ]:
# now we can do:
for epoch in range(epochs): # loop through epochs
        for i in range((n-1)//bs + 1): 
            start_i = i * bs # 0 64
            end_i = start_i + bs # 64 128
            xb = x_train[start_i:end_i]
            yb = y_train[start_i:end_i]
            loss = loss_func(model(xb), yb) # calculate loss on batch
        
            loss.backward() # calculate gradients
            opt.step()
            opt.zero_grad()    

In [ ]:
loss, acc = loss_func(model(xb), yb), accuracy(model(xb), yb)
loss, acc

In [ ]:
# Random test are usefull - so that these models can hint at problems
assert acc>0.7

These tests are written without a rixed random seed to check allow for model variability

#### Dataset 

# code is clunky when looping through mini-batches like this
<code>xb = x_train[start_i:end_i]
yb = y_train[start_i:end_i]</code>

# something like this instead would be more helpful
<code>xb, yb = train_ds[i*bs: i*bs+bs] </code> start_i :(up to) start_i + batch_size for every iteration

In [ ]:
#export
class Dataset():
    def __init__(self, x, y): self.x, self.y = x,y
    def __len__(self): return len(self.x)
    def __getitem__(self, i): return self.x[i], self.y[i]

In [ ]:
train_ds, valid_ds = Dataset(x_train, y_train), Dataset(x_valid, y_valid)
assert len(train_ds) == len(x_train)
assert len(valid_ds) == len(x_valid)

In [ ]:
xb, yb = train_ds[0:5]
assert xb.shape == (5, 28*28)
assert yb.shape == (5,)
xb, yb

In [ ]:
model, opt = get_model()

In [ ]:
for epoch in range(epochs): # loop through epochs
        for i in range((n-1)//bs + 1): 
            xb, yb = train_ds[i*bs : i*bs+bs]
            pred = model(xb)
            loss = loss_func(pred, yb) # calculate loss on batch
        
            loss.backward() # calculate gradients
            opt.step()
            opt.zero_grad()  

In [ ]:
loss, acc = loss_func(model(xb), yb), accuracy(model(xb), yb)
assert acc>0.7
loss, acc

#### Dataloader

We want to change 

<code>for i in range((n-1)//bs + 1): xb, yb = train_ds[i\*bs : i\*bs+bs]</code>

to something more like

<code>for xb, yb in train_dl: ...</code>

In [ ]:
class DataLoader():
    def __init__(self, ds, bs): self.ds, self.bs = ds, bs
    def __iter__(self):
        # range fn moves from from 0 to the length of dataset with batch-size length steps in between
        for i in range(0, len(self.ds), self.bs): yield self.ds[i:i+self.bs]

Yield is a co-routine used for situations when the same item needs to be called many times

In [ ]:
train_dl = DataLoader(train_ds, bs)
valid_dl = DataLoader(valid_ds, bs)

<code>iter</code> generates the co-routine for yield

<code>next</code> grabs the next item from the yield co-routine

In [ ]:
xb, yb = next(iter(valid_dl))
assert xb.shape == (bs, 28*28)
assert yb.shape == (bs, )

In [ ]:
plt.imshow(xb[0].view(28,28))
yb[0]

In [ ]:
model, opt = get_model()

In [ ]:
# @ 1h4 of FastAI course 2019 Lesson 9
def fit():
    for epoch in range(epochs):
        for xb, yb in train_dl:
            pred = model(xb)
            loss = loss_func(pred, yb)
            loss.backward()
            opt.step()
            opt.zero_grad()

In [ ]:
fit()

In [ ]:
loss, acc = loss_func(model(xb), yb), accuracy(model(xb), yb)
assert acc > 0.7
loss, acc

### We have a problem because we are always looping thorugh the training set in order

We'd like the training set to be in a random order, which should differ at each iteration. But the validation set shouldn't be randomized.

In [ ]:
class Sampler():
    def __init__(self, ds, bs, shuffle=False):
        # only the length of the dataset is stored
        self.n, self.bs, self.shuffle = len(ds), bs, shuffle
        
    def __iter__(self):
        self.idxs = torch.randperm(self.n) if self.shuffle else torch.arange(self.n)
        for i in range(0, self.n, self.bs): yield self.idxs[i:i+self.bs]

In [ ]:
len(train_ds[1][0])

In [ ]:
small_ds = Dataset(*train_ds[:10])

In [ ]:
small_ds.shape

In [ ]:
s = Sampler(small_ds, 3, False)
[o for o in s]

In [ ]:
s = Sampler(small_ds, 3, True)
[o for o in s]

In [ ]:
def collate(b):
    xs, ys = zip(*b)
    # stacks tensor list into single list - so that a single "Dataload" can be returned
    return torch.stack(xs),torch.stack(ys)

class DataLoader():
    def __init__(self, ds, sampler, collate_fn=collate):
        self.ds, self.sampler, self.collate_fn = ds, sampler, collate_fn
    # the iter function is calling a co-routine on a co-routine    
    def __iter__(self):
        # grabs all indexes in sample and creates a list and stacks them into a single list of tensors 
        for s in self.sampler: yield self.collate_fn([self.ds[i] for i in s])

In [ ]:
train_samp = Sampler(train_ds, bs, shuffle=True)
valid_samp = Sampler(valid_ds, bs, shuffle=False)

In [ ]:
train_dl = DataLoader(train_ds, train_samp)
valid_dl = DataLoader(train_ds, valid_samp)

In [ ]:
xb, yb = next(iter(valid_dl))
plt.imshow(xb[0].view(28,28))
yb[0]

In [ ]:
xb, yb = next(iter(valid_dl))
plt.imshow(xb[0].view(28,28))
yb[0]

In [ ]:
xb, yb = next(iter(train_dl))
plt.imshow(xb[0].view(28,28))
yb[0]

In [ ]:
xb, yb = next(iter(train_dl))
plt.imshow(xb[0].view(28,28))
yb[0]

### It's returning random numbers

In [ ]:
model, opt = get_model()
fit()

loss, acc = loss_func(model(xb), yb), accuracy(model(xb), yb)
assert acc > 0.7
loss, acc

In [ ]:
#export 
from torch.utils.data  import DataLoader, SequentialSampler, RandomSampler

In [ ]:
# @1h9mins

In [ ]:
train_dl = DataLoader(train_ds, bs, sampler=RandomSampler(train_ds), collate_fn=collate)
valid_dl = DataLoader(valid_ds, bs, sampler=RandomSampler(valid_ds), collate_fn=collate)

In [ ]:
model.opt = get_model()
fit()
loss_func(model(xb), yb), accuracy(model(xb), yb)

In [ ]:
train_dl = DataLoader(train_ds, bs, shuffle=True, drop_last=True)
valid_dl = DataLoader(valid_ds, bs, shuffle=False)

In [ ]:
model.opt = get_model()
fit()
loss, acc = loss_func(model(xb), yb), accuracy(model(xb), yb)
assert acc > 0.7
loss, acc

We didn't implement <code>num_workers</code> which calls multiple threads


### Validation

You should <strong>always</strong> have a validation set as well, in order to identify overfitting

We will calculate and print the validation loss at the end of each epoch.

We need to call <code>model.train()</code> before training and <code>model.eval()</code> before inference to insure the right behavior for <code>nn.BatchNorm2d</code> and <code>nn.Dropout</code>

In [ ]:
def fit(epochs, model, loss_func, opt, train_dl, valid_dl):
    for epoch in range(epochs):
        # Handle batchnorm /dropout
        model.train()
        
        for xb, yb in train_dl:
            loss = loss_func(model(xb), yb)
            loss.backward()
            opt.step()
            opt.zero_grad()
            
        # change to eval mode - turn off batchnorm and dropout during validation set loop
        model.eval()
        
        with torch.no_grad():
            # init loss, accuracy as floats
            tot_loss, tot_acc = 0.,0.
            for xb, yb in valid_dl:
                pred = model(xb)
                tot_loss += loss_func(pred, yb)
                tot_acc  += accuracy(pred, yb)
            nv = len(valid_dl)
            print(epoch, tot_loss/nv, tot_acc/nv)
    return tot_loss/nv, tot_acc/nv        

Note that this will not work very well if batch-size varies, because we'd actually need to be using the weighted averages of the mini-batches, where the weight will depend on the size of the mini-batch

In [ ]:
#export
def get_dls(train_ds, valid_ds, bs, **kwargs):
    return (DataLoader(train_ds, batch_size=bs, shuffle=True, **kwargs),
            # we can double the batch size because we're not doing a backward pass for validation, so no need to store gradients
            DataLoader(valid_ds, batch_size=bs*2, **kwargs))

In [ ]:
# testing
train_dl, valid_dl = get_dls(train_ds, valid_ds, bs)
model, opt = get_model()
loss, acc = fit(5, model, loss_func, opt, train_dl, valid_dl)

In [ ]:
assert acc>0.9

In [ ]:
!python notebook2script.py 03_minibatch_training.ipynb

Basic Training loop

0. setup training data

1. create model
2. calculate loss
3. calculate gradients

Tensorboard works
Check S Gugger. fast progress

In [ ]:

#@1h14min

#### Callbacks

Allow for full customization of each step in the training loop